In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import itertools
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from numpy.random import RandomState
from tqdm import tqdm
import seaborn as sns 

RANDOM = 42
state = RandomState(RANDOM) 

#  Загрузка и подготовка данных

Загрузим данные

In [11]:
zero_frame = pd.read_csv("./datasets/geo_data_0.txt", sep=" ")
first_frame = pd.read_csv("./datasets/geo_data_1.txt", sep=" ")
second_frame = pd.read_csv("./datasets/geo_data_2.txt", sep=" ")

In [12]:
print(zero_frame)

    ind     id         f0         f1        f2     product
0     0  kBEdx -15.001348  -8.276000 -0.005876    3.179103
1     1  62mP7  14.272088  -3.475083  0.999183   26.953261
2     2  vyE1P   6.263187  -5.948386  5.001160  134.766305
3     3  KcrkZ -13.081196 -11.506057  4.999415  137.945408
4     4  AHL4O  12.702195  -8.147433  5.004363  134.766305
5     5  HHckp  -3.327590  -2.205276  3.003647   84.038886
6     6  h5Ujo -11.142655 -10.133399  4.002382  110.992147
7     7  muH9x   4.234715  -0.001354  2.004588   53.906522
8     8  YiRkx  13.355129  -0.332068  4.998647  134.766305
9     9  jG6Gi   1.069227 -11.025667  4.997844  137.945408
10   10  7uHUe  11.777049  -5.334084  2.003033   53.906522
11   11  OXyvW  16.320755  -0.562946 -0.001783    0.000000
12   12  cB8re   7.736313  -6.093374  3.982531  107.813044
13   13  igmai   6.695604  -0.749449 -0.007630    0.000000
14   14  3OdkE -10.985487  -5.605994  2.991130   84.038886
15   15  3tVUO  -0.347599  -6.275884 -0.003448    3.1791

# Первичное обработка данных


In [31]:
print(zero_frame.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   ind      40 non-null     int64  
 1   id       40 non-null     object 
 2   f0       40 non-null     float64
 3   f1       40 non-null     float64
 4   f2       40 non-null     float64
 5   product  40 non-null     float64
dtypes: float64(4), int64(1), object(1)
memory usage: 2.0+ KB
None


## Проверка на дубликаты

In [32]:
print(f'Количество дубликатов:{zero_frame.duplicated().sum()}')
print(f'Количество дубликатов:{first_frame.duplicated().sum()}')
print(f'Количество дубликатов:{second_frame.duplicated().sum()}')

Количество дубликатов:0
Количество дубликатов:0
Количество дубликатов:0


In [58]:
def find_duplicates(frame):
    row_count = len(zero_frame.index)
    dupls = 0
    
    for index, row in zero_frame.iterrows():
        for i in range(index+1, row_count):
            if row.equals(zero_frame.iloc[i]):
                dupls+=1
                print(row)
                print(zero_frame.iloc[i])
    if not dupls:
        print("No duplicates")

In [59]:
find_duplicates(zero_frame)

No duplicates


## Поиск медианы

In [89]:
def find_mediana(frame, column):
    test = frame[column]
    res = test.sort_values(ascending=True)
    
    res = res.reset_index(drop=True)
    
    pos = len(frame)/2
    ans = res[pos]
    #print(res)
    return ans

In [90]:
find_mediana(zero_frame, 'product')

-4.729495

## Интерквартильный размах

In [145]:
def quantiles(frame, column):
    test_1 = frame[column]
    res_1 = test_1.sort_values(ascending=True)
    res_1 = res_1.reset_index(drop=True)
    
    left_half = res_1.iloc[0:int(len(frame)/2)]
    right_half = res_1.iloc[int(len(frame)/2):-1]
    
    q1 = left_half.median()
    q3 = right_half.median()
    
    iqr = q3 - q1
    return [q1, q3, iqr]

In [146]:
ans = quantiles(zero_frame, 'product')
q1 = ans[0]
q3 = ans[1]
iqr = ans[2]
print(q1, q3, iqr)
lower_limit = q1 - 1.5 * iqr
high_limit = q3 + 1,5 * iqr
print(lower_limit, high_limit)

53.906522 110.992147 57.085625
-31.7219155 (111.992147, 285.428125)


### Выводы

Заметим, что в некоторых столбцах есть данные, выходящие за границы. Эти данные будут мешать модели обучаться, отвлекая её. Поэтому их необходимо удалить.
Выпишем необходимые столбцы для обработки в формате - (фрейм, столбец):
1. (zero_frame,"f2")
1. (first_frame,"f1")
1. (second_frame,"f0")
1. (second_frame,"f1")
1. (second_frame,"f2")

## Исправим недочеты, найденные в данных 

In [9]:
def remove_ouliers(frame,column):
    q25=np.array(frame[column].quantile(0.25))
    
    q75=np.array(frame[column].quantile(0.75))
    first_part=q25-1.5*(q75-q25)
    second_part=q75+1.5*(q75-q25)
    del_index = []
    for index_value, value in zip(frame[column].index,frame[column]):
        if second_part <= value or value <= first_part:
            del_index.append(index_value)
    
    print('Количество строк, выбранных для удаления: ',len(del_index))
    return del_index

In [10]:
noise_data = [(zero_frame,"f2"),
 (first_frame,"f1"),
 (second_frame,"f0"),
 (second_frame,"f1"),
 (second_frame,"f2")]

for frame,column in noise_data:
    indexes = remove_ouliers(frame,column)
    frame.drop(indexes,axis = 0,inplace = True)

Количество строк, выбранных для удаления:  506
Количество строк, выбранных для удаления:  636
Количество строк, выбранных для удаления:  733
Количество строк, выбранных для удаления:  682
Количество строк, выбранных для удаления:  582


Выборки пострадали не сильно, в первых двух потери составили менее 1%, в последней же потеря данных составила 2%

In [11]:
zero_frame = zero_frame.reset_index(drop = True)
first_frame = first_frame.reset_index(drop = True)
second_frame = second_frame.reset_index(drop = True)

## Проверим корреляцию между признаками

In [12]:
display_info(True,
             Zero_Frame = {'':zero_frame.corr()},
             First_Frame = {'':first_frame.corr()},
             Second_Frame = {'':second_frame.corr()})

##################
#   Zero_Frame   #
##################



,f0,f1,f2,product
f0,1.000000,-0.440716,-0.003339,0.143591
f1,-0.440716,1.000000,0.001098,-0.192748
f2,-0.003339,0.001098,1.000000,0.480159
product,0.143591,-0.192748,0.480159,1.000000



###################
#   First_Frame   #
###################



,f0,f1,f2,product
f0,1.000000,0.178877,-0.001670,-0.030389
f1,0.178877,1.000000,-0.002756,-0.010174
f2,-0.001670,-0.002756,1.000000,0.999397
product,-0.030389,-0.010174,0.999397,1.000000



####################
#   Second_Frame   #
####################



,f0,f1,f2,product
f0,1.000000,0.000649,0.000473,-0.002962
f1,0.000649,1.000000,0.000710,-0.000895
f2,0.000473,0.000710,1.000000,0.441211
product,-0.002962,-0.000895,0.441211,1.000000


### Вывод

Заметим, что в нулевом фрейме признаки f0 и f1 отрицательно коррелируют относительно друг друга и f2 слабо положительно коррелирует с целевым признаком. Так же в первом фрейме очень сильно коррелирует целевой признак и f2. Во втором фрейме так же есть коррелирующие признаки, такие как f2 и product.

Если в случае с первым фреймом все достаточно понятно, там очень высокая корреляция и признак f2 следует удалить, то вот в случае с другими двумя выборками стоит опираться на результат, который мы получим на моделях, Следовательно необходимо подготовить 3 выборок:
1. Нулевая со всеми столбцами
1. Первая без f2
1. Вторая со всеми столбцами